In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import io


In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
df = pd.read_csv('../data/df_completo.csv')
df.head(3)

,id,Event,Site,Date,White,Black,Result,BlackElo,ECO,WhiteElo,...,B<=25,B<=50,B<=100,B<=200,B<=500,B>500,Wprojection,Bprojection,DifProjection,Plays
0,0,New York,New York,1918,"Morrison, John Stuart","Marshall, Frank James",0-1,NaN,C89,NaN,...,18,10,6,4,2,0,1152,-38380,-37228,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
1,1168,New York,New York,1918,"Capablanca, Jose Raul","Marshall, Frank James",1-0,NaN,C89,NaN,...,5,7,3,4,0,1,17855,0,17855,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...
2,1772,Brno,Brno,1921,"Sery, S.","Vecsey, Zoltan",1-0,NaN,C89,NaN,...,5,0,1,0,0,1,3851,0,3851,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...


In [4]:
def get_move_index(n, turn='w'):
    if turn.lower() == 'b':
        return 2 * n - 1
    else:
        return 2 * n - 1 - 1

In [5]:
def get_subvariation(row, move_numbers:list):
    # fen = 'r1bq1rk1/2p1bppp/p1n2n2/1p1pp3/4P3/1BP2N2/PP1P1PPP/RNBQR1K1 w - - 0 9'
    # board = chess.Board(fen=fen)
    # white_move = 2 * move_number - 1 - 1
    # black_move = 2 * move_number - 1

    pgn = io.StringIO(row['Plays'])

    game = chess.pgn.read_game(pgn)

    mainline = [m.san() for m in game.mainline()]

    s = ''

    for m_num in move_numbers:
        if len(mainline) > get_move_index(m_num, turn='b'):
            s += '{0}. {1} {2} '.format(
                m_num,
                mainline[get_move_index(m_num, turn='w')],
                mainline[get_move_index(m_num, turn='b')]
            )

    row['SubVariation'] = s[:-1]

    row['Mainline'] = mainline
    
    return row

df = df.apply(lambda row: get_subvariation(row, [9,10]), axis=1)

In [6]:
df.head(1)

,id,Event,Site,Date,White,Black,Result,BlackElo,ECO,WhiteElo,...,B<=100,B<=200,B<=500,B>500,Wprojection,Bprojection,DifProjection,Plays,SubVariation,Mainline
0,0,New York,New York,1918,"Morrison, John Stuart","Marshall, Frank James",0-1,NaN,C89,NaN,...,6,4,2,0,1152,-38380,-37228,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,9. exd5 Nxd5 10. d4 exd4,"[e4, e5, Nf3, Nc6, Bb5, a6, Ba4, Nf6, O-O, Be7..."


In [7]:
df['SubVariation'].unique().tolist()

['9. exd5 Nxd5 10. d4 exd4',
 '9. exd5 Nxd5 10. Nxe5 Nxe5',
 '9. exd5 e4 10. Ng5 Ne5',
 '9. exd5 e4 10. dxc6 exf3',
 '9. exd5 Nxd5 10. a4 Bb7',
 '9. d3 Qd6 10. Nbd2 Be6',
 '9. d3 dxe4 10. dxe4 Qxd1',
 '9. d4 Nxe4 10. dxe5 Be6',
 '9. d3 d4 10. Qe2 Bg4',
 '9. d4 exd4 10. e5 Ne4',
 '9. exd5 Nxd5 10. h3 Bb7',
 '9. d4 dxe4 10. Nxe5 Bb7',
 '9. exd5 Nxd5 10. Bc2 Bg4',
 '9. exd5 e4 10. Rxe4 Nxe4',
 '9. exd5 Nxd5 10. d3 Be6',
 '9. exd5 Nxd5 10. d3 Bg4',
 '9. d4 Bg4 10. exd5 e4',
 '9. exd5 Nxd5 10. d3 Bf6',
 '9. exd5 Nxd5 10. d3 Bb7',
 '9. d3 Bg4 10. Bg5 dxe4',
 '9. d4 Nxe4 10. Nxe5 Nxe5',
 '9. exd5 Nxd5 10. a4 Rb8',
 '9. exd5 Nxd5 10. h3 Be6',
 '9. d3 Bg4 10. Nbd2 dxe4',
 '9. exd5 e4 10. Ng5 Bg4',
 '9. d4 dxe4 10. Nxe5 Na5',
 '9. d4 exd4 10. exd5 Nxd5',
 '9. d3 d4 10. a4 b4',
 '9. d3 Bb7 10. Nbd2 dxe4',
 '9. d3 Bg4 10. h3 Bh5',
 '9. exd5 Nxd5',
 '9. h3 Bb7 10. d3 h6',
 '9. exd5 e4 10. Ng5 Nxd5',
 '9. exd5 Nxd5 10. Qe2 Bb7',
 '9. Bc2 dxe4 10. Bxe4 Nxe4',
 '9. exd5 e4 10. d4 exf3',
 '9. d3 h6 10.

In [8]:
df.to_csv('../data/df_subvar_9-10.csv', index=False)